In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense,Embedding,LSTM,Dropout
import re

In [11]:
df=pd.read_csv(r'C:\Users\Ganesh\Desktop\jupiter\neuron\Project\consumer_complaints.csv',low_memory=False)

In [12]:
# cheacking the data size
df.shape

(555957, 18)

In [13]:
df.head()

,date_received,product,sub_product,issue,sub_issue,consumer_complaint_narrative,company_public_response,company,state,zipcode,tags,consumer_consent_provided,submitted_via,date_sent_to_company,company_response_to_consumer,timely_response,consumer_disputed?,complaint_id
0,08/30/2013,Mortgage,Other mortgage,"Loan modification,collection,foreclosure",NaN,NaN,NaN,U.S. Bancorp,CA,95993,NaN,NaN,Referral,09/03/2013,Closed with explanation,Yes,Yes,511074
1,08/30/2013,Mortgage,Other mortgage,"Loan servicing, payments, escrow account",NaN,NaN,NaN,Wells Fargo & Company,CA,91104,NaN,NaN,Referral,09/03/2013,Closed with explanation,Yes,Yes,511080
2,08/30/2013,Credit reporting,NaN,Incorrect information on credit report,Account status,NaN,NaN,Wells Fargo & Company,NY,11764,NaN,NaN,Postal mail,09/18/2013,Closed with explanation,Yes,No,510473
3,08/30/2013,Student loan,Non-federal student loan,Repaying your loan,Repaying your loan,NaN,NaN,"Navient Solutions, Inc.",MD,21402,NaN,NaN,Email,08/30/2013,Closed with explanation,Yes,Yes,510326
4,08/30/2013,Debt collection,Credit card,False statements or representation,Attempted to collect wrong amount,NaN,NaN,Resurgent Capital Services L.P.,GA,30106,NaN,NaN,Web,08/30/2013,Closed with explanation,Yes,Yes,511067


In [149]:
# cheacking null values
df.isnull().sum()

date_received                        0
product                              0
sub_product                     158322
issue                                0
sub_issue                       343335
consumer_complaint_narrative    489151
company_public_response         470833
company                              0
state                             4887
zipcode                           4505
tags                            477998
consumer_consent_provided       432499
submitted_via                        0
date_sent_to_company                 0
company_response_to_consumer         0
timely_response                      0
consumer_disputed?                   0
complaint_id                         0
dtype: int64

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 555957 entries, 0 to 555956
Data columns (total 18 columns):
 #   Column                        Non-Null Count   Dtype 
---  ------                        --------------   ----- 
 0   date_received                 555957 non-null  object
 1   product                       555957 non-null  object
 2   sub_product                   397635 non-null  object
 3   issue                         555957 non-null  object
 4   sub_issue                     212622 non-null  object
 5   consumer_complaint_narrative  66806 non-null   object
 6   company_public_response       85124 non-null   object
 7   company                       555957 non-null  object
 8   state                         551070 non-null  object
 9   zipcode                       551452 non-null  object
 10  tags                          77959 non-null   object
 11  consumer_consent_provided     123458 non-null  object
 12  submitted_via                 555957 non-null  object
 13 

In [81]:
data=df.copy()

In [128]:
data.columns

Index(['date_received', 'product', 'sub_product', 'issue', 'sub_issue',
       'consumer_complaint_narrative', 'company_public_response', 'company',
       'state', 'zipcode', 'tags', 'consumer_consent_provided',
       'submitted_via', 'date_sent_to_company', 'company_response_to_consumer',
       'timely_response', 'consumer_disputed?', 'complaint_id'],
      dtype='object')

In [150]:
# we remove the columns which not requried
final=data.iloc[:,1:5]

In [151]:
final.head()

,product,sub_product,issue,sub_issue
0,Mortgage,Other mortgage,"Loan modification,collection,foreclosure",NaN
1,Mortgage,Other mortgage,"Loan servicing, payments, escrow account",NaN
2,Credit reporting,NaN,Incorrect information on credit report,Account status
3,Student loan,Non-federal student loan,Repaying your loan,Repaying your loan
4,Debt collection,Credit card,False statements or representation,Attempted to collect wrong amount


In [152]:
final['product'].value_counts()

Mortgage                   186475
Debt collection            101052
Credit reporting            91854
Credit card                 66468
Bank account or service     62563
Consumer Loan               20990
Student loan                15839
Payday loan                  3877
Money transfers              3812
Prepaid card                 2470
Other financial service       557
Name: product, dtype: int64

In [153]:
# we remove financial services as its have very few entrie
final=final.drop(final[final['product']=='Other financial service'].index)

In [154]:
final['product'].value_counts()

Mortgage                   186475
Debt collection            101052
Credit reporting            91854
Credit card                 66468
Bank account or service     62563
Consumer Loan               20990
Student loan                15839
Payday loan                  3877
Money transfers              3812
Prepaid card                 2470
Name: product, dtype: int64

In [155]:
# we Consolidate the credid card and preepaid card into credit card
final['product']=final['product'].replace(['Prepaid card'],'Credit card')

In [160]:
final['product'].value_counts()

Mortgage                   186475
Debt collection            101052
Credit reporting            91854
Credit card                 68938
Bank account or service     62563
Consumer Loan               20990
Student loan                15839
Payday loan                  3877
Money transfers              3812
Name: product, dtype: int64

In [161]:
final.isnull().sum()

product             0
sub_product    158322
issue               0
sub_issue      342778
dtype: int64

In [162]:
d={'product':final['product'],'issue':final['issue']}

In [163]:
# considering one isuue ins independent and product is dependent variable to be predicted
new_data=pd.DataFrame(data=d)

In [164]:
new_data.head()

,product,issue
0,Mortgage,"Loan modification,collection,foreclosure"
1,Mortgage,"Loan servicing, payments, escrow account"
2,Credit reporting,Incorrect information on credit report
3,Student loan,Repaying your loan
4,Debt collection,False statements or representation


In [178]:
# reset index 
new_data=new_data.reset_index(drop=True)

In [181]:
new_data.head()

,product,issue
0,Mortgage,"Loan modification,collection,foreclosure"
1,Mortgage,"Loan servicing, payments, escrow account"
2,Credit reporting,Incorrect information on credit report
3,Student loan,Repaying your loan
4,Debt collection,False statements or representation


In [182]:
Y=new_data['product']
X=new_data['issue']

In [192]:
# spliting data train and test data set since dataset is very large we split two time so can run on normal machine
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest=train_test_split(X,Y,test_size=0.70,random_state=42)

In [194]:
xc_train,xc_test,yc_train,yc_test=train_test_split(xtrain,ytrain,test_size=0.25,random_state=42)

In [195]:
xc_train.shape

(124965,)

In [262]:
yc_train.shape

(124965,)

In [198]:
l=list(xc_train)

In [199]:
l[1]

'Loan servicing, payments, escrow account'

In [200]:
# data preprocessing
from tqdm import tqdm
corpus=[]

for i in tqdm(range(len(l))):
    issue=re.sub('[^a-zA-Z]',' ',l[i])
    issue=issue.lower()
    issue=issue.split()
    issue=[x for x in issue if not x in stopwords.words('english')]
    issue=' '.join(issue)
    corpus.append(issue)

100%|█████████████████████████████████████████████████████████████████████████| 124965/124965 [04:09<00:00, 500.79it/s]


In [263]:
yc_train.value_counts()

Mortgage                   42012
Debt collection            22741
Credit reporting           20594
Credit card                15490
Bank account or service    14046
Consumer Loan               4718
Student loan                3618
Money transfers              887
Payday loan                  859
Name: product, dtype: int64

In [264]:
len(corpus)

124965

In [265]:
len(yc_train)

124965

In [266]:
# creatind dummy columns for each catogory of product
yc_train=pd.get_dummies(yc_train,drop_first=True)

In [267]:
yc_train.shape

(124965, 8)

In [268]:
# defining dictionary size
vec_size=5000
# defined sentence length
sen_length=20

In [270]:
# crating onehot representation
from tensorflow.keras.preprocessing.sequence import pad_sequences 
from tensorflow.keras.preprocessing.text import one_hot
onehot=[one_hot(words,vec_size) for words in corpus]

In [271]:
len(onehot)

124965

In [272]:
# to get equal length we use padding
seq_pad=pad_sequences(onehot,padding='pre',maxlen=20)
print(seq_pad)

[[   0    0    0 ...   53 4918 1592]
 [   0    0    0 ... 3216 1892 2600]
 [   0    0    0 ...  593 2001 4626]
 ...
 [   0    0    0 ... 2001 3921 1991]
 [   0    0    0 ...  593 2001 4626]
 [   0    0    0 ... 2227 2034 1736]]


In [273]:
len(seq_pad)

124965

In [274]:
# define the feature size i.e diamention
dim=40

In [275]:
# initialize the model
model=Sequential()
model.add(Embedding(vec_size,dim,input_length=sen_length))
model.add(LSTM(100))
model.add(Dense(8,activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
lstm_4 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dense_4 (Dense)              (None, 8)                 808       
Total params: 257,208
Trainable params: 257,208
Non-trainable params: 0
_________________________________________________________________
None


In [276]:
len(onehot),yc_train.shape

(124965, (124965, 8))

In [277]:
# convert onehot into array
import numpy as np
X_final=np.array(seq_pad)
y_final=np.array(yc_train)

In [278]:
X_final.shape,y_final.shape

((124965, 20), (124965, 8))

In [279]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.30, random_state=20)

In [280]:
X_train.shape,y_train.shape

((87475, 20), (87475, 8))

In [281]:
X_test.shape,y_test.shape

((37490, 20), (37490, 8))

In [282]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=5,batch_size=64)

Train on 87475 samples, validate on 37490 samples
Epoch 1/5


11776/87475 [===>..........................] - ETA: 10:59:01 - loss: 1.9163 - accuracy: 0.12 - ETA: 5:30:21 - loss: 1.9098 - accuracy: 0.2812 - ETA: 3:40:43 - loss: 1.8526 - accuracy: 0.286 - ETA: 2:45:51 - loss: 1.8449 - accuracy: 0.300 - ETA: 2:12:53 - loss: 1.8245 - accuracy: 0.321 - ETA: 1:50:53 - loss: 1.7970 - accuracy: 0.338 - ETA: 1:35:09 - loss: 1.7980 - accuracy: 0.343 - ETA: 1:23:29 - loss: 1.7911 - accuracy: 0.335 - ETA: 1:14:18 - loss: 1.7782 - accuracy: 0.335 - ETA: 1:06:58 - loss: 1.7721 - accuracy: 0.331 - ETA: 1:00:56 - loss: 1.7634 - accuracy: 0.328 - ETA: 55:56 - loss: 1.7558 - accuracy: 0.3229  - ETA: 51:41 - loss: 1.7446 - accuracy: 0.325 - ETA: 48:03 - loss: 1.7276 - accuracy: 0.327 - ETA: 44:59 - loss: 1.7226 - accuracy: 0.325 - ETA: 42:13 - loss: 1.7020 - accuracy: 0.329 - ETA: 39:46 - loss: 1.6920 - accuracy: 0.330 - ETA: 37:36 - loss: 1.6815 - accuracy: 0.329 - ETA: 35:40 - loss: 1.6657 - accuracy: 0.332 - ETA: 33:56 - loss: 1.6535 - accuracy: 0.328 - ETA: 32:

23680/87475 [=======>......................] - ETA: 4:16 - loss: 0.7311 - accuracy: 0.66 - ETA: 4:15 - loss: 0.7287 - accuracy: 0.66 - ETA: 4:14 - loss: 0.7258 - accuracy: 0.66 - ETA: 4:13 - loss: 0.7232 - accuracy: 0.66 - ETA: 4:12 - loss: 0.7202 - accuracy: 0.66 - ETA: 4:11 - loss: 0.7179 - accuracy: 0.66 - ETA: 4:09 - loss: 0.7154 - accuracy: 0.66 - ETA: 4:08 - loss: 0.7127 - accuracy: 0.66 - ETA: 4:07 - loss: 0.7101 - accuracy: 0.66 - ETA: 4:06 - loss: 0.7078 - accuracy: 0.66 - ETA: 4:05 - loss: 0.7053 - accuracy: 0.67 - ETA: 4:04 - loss: 0.7030 - accuracy: 0.67 - ETA: 4:03 - loss: 0.7009 - accuracy: 0.67 - ETA: 4:02 - loss: 0.6986 - accuracy: 0.67 - ETA: 4:00 - loss: 0.6965 - accuracy: 0.67 - ETA: 3:59 - loss: 0.6941 - accuracy: 0.67 - ETA: 3:58 - loss: 0.6921 - accuracy: 0.67 - ETA: 3:57 - loss: 0.6893 - accuracy: 0.67 - ETA: 3:56 - loss: 0.6871 - accuracy: 0.67 - ETA: 3:55 - loss: 0.6846 - accuracy: 0.67 - ETA: 3:54 - loss: 0.6822 - accuracy: 0.67 - ETA: 3:53 - loss: 0.6802 - ac

36032/87475 [===========>..................] - ETA: 2:18 - loss: 0.4481 - accuracy: 0.75 - ETA: 2:18 - loss: 0.4473 - accuracy: 0.75 - ETA: 2:17 - loss: 0.4464 - accuracy: 0.75 - ETA: 2:17 - loss: 0.4455 - accuracy: 0.76 - ETA: 2:16 - loss: 0.4445 - accuracy: 0.76 - ETA: 2:16 - loss: 0.4436 - accuracy: 0.76 - ETA: 2:16 - loss: 0.4428 - accuracy: 0.76 - ETA: 2:15 - loss: 0.4417 - accuracy: 0.76 - ETA: 2:15 - loss: 0.4408 - accuracy: 0.76 - ETA: 2:15 - loss: 0.4401 - accuracy: 0.76 - ETA: 2:14 - loss: 0.4394 - accuracy: 0.76 - ETA: 2:14 - loss: 0.4386 - accuracy: 0.76 - ETA: 2:14 - loss: 0.4377 - accuracy: 0.76 - ETA: 2:13 - loss: 0.4369 - accuracy: 0.76 - ETA: 2:13 - loss: 0.4359 - accuracy: 0.76 - ETA: 2:13 - loss: 0.4350 - accuracy: 0.76 - ETA: 2:12 - loss: 0.4341 - accuracy: 0.76 - ETA: 2:12 - loss: 0.4333 - accuracy: 0.76 - ETA: 2:12 - loss: 0.4325 - accuracy: 0.76 - ETA: 2:11 - loss: 0.4316 - accuracy: 0.76 - ETA: 2:11 - loss: 0.4306 - accuracy: 0.76 - ETA: 2:11 - loss: 0.4297 - ac

48256/87475 [===============>..............] - ETA: 1:29 - loss: 0.3288 - accuracy: 0.79 - ETA: 1:28 - loss: 0.3284 - accuracy: 0.79 - ETA: 1:28 - loss: 0.3282 - accuracy: 0.79 - ETA: 1:28 - loss: 0.3279 - accuracy: 0.79 - ETA: 1:28 - loss: 0.3274 - accuracy: 0.79 - ETA: 1:28 - loss: 0.3271 - accuracy: 0.79 - ETA: 1:27 - loss: 0.3267 - accuracy: 0.79 - ETA: 1:27 - loss: 0.3263 - accuracy: 0.79 - ETA: 1:27 - loss: 0.3257 - accuracy: 0.79 - ETA: 1:27 - loss: 0.3254 - accuracy: 0.79 - ETA: 1:27 - loss: 0.3249 - accuracy: 0.79 - ETA: 1:27 - loss: 0.3245 - accuracy: 0.79 - ETA: 1:26 - loss: 0.3240 - accuracy: 0.79 - ETA: 1:26 - loss: 0.3236 - accuracy: 0.79 - ETA: 1:26 - loss: 0.3231 - accuracy: 0.79 - ETA: 1:26 - loss: 0.3228 - accuracy: 0.79 - ETA: 1:26 - loss: 0.3225 - accuracy: 0.79 - ETA: 1:25 - loss: 0.3216 - accuracy: 0.79 - ETA: 1:25 - loss: 0.3211 - accuracy: 0.79 - ETA: 1:25 - loss: 0.3206 - accuracy: 0.79 - ETA: 1:25 - loss: 0.3204 - accuracy: 0.79 - ETA: 1:24 - loss: 0.3201 - ac

...

39296/87475 [============>.................] - ETA: 53s - loss: 0.0286 - accuracy: 0.874 - ETA: 52s - loss: 0.0286 - accuracy: 0.874 - ETA: 52s - loss: 0.0286 - accuracy: 0.874 - ETA: 52s - loss: 0.0286 - accuracy: 0.874 - ETA: 52s - loss: 0.0285 - accuracy: 0.874 - ETA: 52s - loss: 0.0288 - accuracy: 0.874 - ETA: 52s - loss: 0.0288 - accuracy: 0.874 - ETA: 52s - loss: 0.0288 - accuracy: 0.874 - ETA: 52s - loss: 0.0288 - accuracy: 0.874 - ETA: 52s - loss: 0.0288 - accuracy: 0.874 - ETA: 52s - loss: 0.0289 - accuracy: 0.874 - ETA: 52s - loss: 0.0290 - accuracy: 0.874 - ETA: 52s - loss: 0.0290 - accuracy: 0.874 - ETA: 52s - loss: 0.0290 - accuracy: 0.873 - ETA: 52s - loss: 0.0290 - accuracy: 0.873 - ETA: 52s - loss: 0.0289 - accuracy: 0.873 - ETA: 52s - loss: 0.0289 - accuracy: 0.873 - ETA: 51s - loss: 0.0289 - accuracy: 0.874 - ETA: 51s - loss: 0.0289 - accuracy: 0.874 - ETA: 51s - loss: 0.0289 - accuracy: 0.873 - ETA: 51s - loss: 0.0289 - accuracy: 0.873 - ETA: 51s - loss: 0.0289 - acc

52224/87475 [================>.............] - ETA: 41s - loss: 0.0282 - accuracy: 0.876 - ETA: 41s - loss: 0.0282 - accuracy: 0.876 - ETA: 41s - loss: 0.0282 - accuracy: 0.876 - ETA: 41s - loss: 0.0283 - accuracy: 0.876 - ETA: 41s - loss: 0.0282 - accuracy: 0.876 - ETA: 41s - loss: 0.0282 - accuracy: 0.876 - ETA: 41s - loss: 0.0282 - accuracy: 0.876 - ETA: 41s - loss: 0.0281 - accuracy: 0.876 - ETA: 41s - loss: 0.0281 - accuracy: 0.876 - ETA: 40s - loss: 0.0281 - accuracy: 0.876 - ETA: 40s - loss: 0.0281 - accuracy: 0.876 - ETA: 40s - loss: 0.0281 - accuracy: 0.876 - ETA: 40s - loss: 0.0281 - accuracy: 0.876 - ETA: 40s - loss: 0.0281 - accuracy: 0.876 - ETA: 40s - loss: 0.0281 - accuracy: 0.876 - ETA: 40s - loss: 0.0280 - accuracy: 0.876 - ETA: 40s - loss: 0.0281 - accuracy: 0.876 - ETA: 40s - loss: 0.0281 - accuracy: 0.876 - ETA: 40s - loss: 0.0280 - accuracy: 0.876 - ETA: 40s - loss: 0.0280 - accuracy: 0.876 - ETA: 40s - loss: 0.0280 - accuracy: 0.876 - ETA: 40s - loss: 0.0280 - acc

65152/87475 [=====================>........] - ETA: 30s - loss: 0.0276 - accuracy: 0.876 - ETA: 30s - loss: 0.0277 - accuracy: 0.876 - ETA: 30s - loss: 0.0277 - accuracy: 0.876 - ETA: 30s - loss: 0.0277 - accuracy: 0.876 - ETA: 29s - loss: 0.0276 - accuracy: 0.876 - ETA: 29s - loss: 0.0277 - accuracy: 0.876 - ETA: 29s - loss: 0.0277 - accuracy: 0.876 - ETA: 29s - loss: 0.0276 - accuracy: 0.876 - ETA: 29s - loss: 0.0276 - accuracy: 0.876 - ETA: 29s - loss: 0.0276 - accuracy: 0.876 - ETA: 29s - loss: 0.0276 - accuracy: 0.876 - ETA: 29s - loss: 0.0276 - accuracy: 0.876 - ETA: 29s - loss: 0.0276 - accuracy: 0.876 - ETA: 29s - loss: 0.0276 - accuracy: 0.876 - ETA: 29s - loss: 0.0276 - accuracy: 0.876 - ETA: 29s - loss: 0.0277 - accuracy: 0.876 - ETA: 29s - loss: 0.0277 - accuracy: 0.876 - ETA: 29s - loss: 0.0277 - accuracy: 0.876 - ETA: 29s - loss: 0.0277 - accuracy: 0.876 - ETA: 29s - loss: 0.0277 - accuracy: 0.876 - ETA: 29s - loss: 0.0277 - accuracy: 0.876 - ETA: 28s - loss: 0.0276 - acc

77632/87475 [=========================>....] - ETA: 19s - loss: 0.0272 - accuracy: 0.875 - ETA: 19s - loss: 0.0272 - accuracy: 0.875 - ETA: 18s - loss: 0.0272 - accuracy: 0.875 - ETA: 18s - loss: 0.0272 - accuracy: 0.875 - ETA: 18s - loss: 0.0272 - accuracy: 0.875 - ETA: 18s - loss: 0.0272 - accuracy: 0.875 - ETA: 18s - loss: 0.0272 - accuracy: 0.875 - ETA: 18s - loss: 0.0272 - accuracy: 0.875 - ETA: 18s - loss: 0.0272 - accuracy: 0.875 - ETA: 18s - loss: 0.0272 - accuracy: 0.875 - ETA: 18s - loss: 0.0272 - accuracy: 0.875 - ETA: 18s - loss: 0.0272 - accuracy: 0.875 - ETA: 18s - loss: 0.0271 - accuracy: 0.875 - ETA: 18s - loss: 0.0272 - accuracy: 0.875 - ETA: 18s - loss: 0.0272 - accuracy: 0.875 - ETA: 18s - loss: 0.0271 - accuracy: 0.875 - ETA: 18s - loss: 0.0272 - accuracy: 0.875 - ETA: 18s - loss: 0.0272 - accuracy: 0.875 - ETA: 18s - loss: 0.0272 - accuracy: 0.875 - ETA: 17s - loss: 0.0272 - accuracy: 0.875 - ETA: 17s - loss: 0.0272 - accuracy: 0.875 - ETA: 17s - loss: 0.0272 - acc

87475/87475 [==============================] - ETA: 8s - loss: 0.0269 - accuracy: 0.87 - ETA: 8s - loss: 0.0269 - accuracy: 0.87 - ETA: 8s - loss: 0.0269 - accuracy: 0.87 - ETA: 8s - loss: 0.0269 - accuracy: 0.87 - ETA: 8s - loss: 0.0269 - accuracy: 0.87 - ETA: 8s - loss: 0.0269 - accuracy: 0.87 - ETA: 8s - loss: 0.0269 - accuracy: 0.87 - ETA: 8s - loss: 0.0269 - accuracy: 0.87 - ETA: 7s - loss: 0.0268 - accuracy: 0.87 - ETA: 7s - loss: 0.0268 - accuracy: 0.87 - ETA: 7s - loss: 0.0269 - accuracy: 0.87 - ETA: 7s - loss: 0.0269 - accuracy: 0.87 - ETA: 7s - loss: 0.0269 - accuracy: 0.87 - ETA: 7s - loss: 0.0269 - accuracy: 0.87 - ETA: 7s - loss: 0.0269 - accuracy: 0.87 - ETA: 7s - loss: 0.0269 - accuracy: 0.87 - ETA: 7s - loss: 0.0269 - accuracy: 0.87 - ETA: 7s - loss: 0.0269 - accuracy: 0.87 - ETA: 7s - loss: 0.0269 - accuracy: 0.87 - ETA: 7s - loss: 0.0269 - accuracy: 0.87 - ETA: 7s - loss: 0.0269 - accuracy: 0.87 - ETA: 7s - loss: 0.0269 - accuracy: 0.87 - ETA: 7s - loss: 0.0268 - accu

In [289]:
#from sklearn.metrics import accuracy_Score

accr = model.evaluate(X_test,y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))


28128/37490 [=====================>........] - ETA: 19s - loss: 0.0485 - accuracy: 0.906 - ETA: 28s - loss: 0.0155 - accuracy: 0.882 - ETA: 19s - loss: 0.0132 - accuracy: 0.881 - ETA: 17s - loss: 0.0185 - accuracy: 0.868 - ETA: 15s - loss: 0.0215 - accuracy: 0.860 - ETA: 15s - loss: 0.0180 - accuracy: 0.872 - ETA: 14s - loss: 0.0212 - accuracy: 0.866 - ETA: 14s - loss: 0.0271 - accuracy: 0.866 - ETA: 14s - loss: 0.0255 - accuracy: 0.871 - ETA: 13s - loss: 0.0239 - accuracy: 0.872 - ETA: 13s - loss: 0.0245 - accuracy: 0.874 - ETA: 13s - loss: 0.0256 - accuracy: 0.873 - ETA: 14s - loss: 0.0264 - accuracy: 0.874 - ETA: 14s - loss: 0.0259 - accuracy: 0.876 - ETA: 14s - loss: 0.0266 - accuracy: 0.874 - ETA: 13s - loss: 0.0276 - accuracy: 0.874 - ETA: 13s - loss: 0.0270 - accuracy: 0.874 - ETA: 13s - loss: 0.0286 - accuracy: 0.873 - ETA: 13s - loss: 0.0282 - accuracy: 0.873 - ETA: 13s - loss: 0.0274 - accuracy: 0.875 - ETA: 13s - loss: 0.0266 - accuracy: 0.875 - ETA: 13s - loss: 0.0264 - acc

37490/37490 [==============================] - ETA: 3s - loss: 0.0264 - accuracy: 0.87 - ETA: 3s - loss: 0.0265 - accuracy: 0.87 - ETA: 3s - loss: 0.0265 - accuracy: 0.87 - ETA: 3s - loss: 0.0265 - accuracy: 0.87 - ETA: 3s - loss: 0.0265 - accuracy: 0.87 - ETA: 3s - loss: 0.0265 - accuracy: 0.87 - ETA: 3s - loss: 0.0265 - accuracy: 0.87 - ETA: 3s - loss: 0.0265 - accuracy: 0.87 - ETA: 3s - loss: 0.0265 - accuracy: 0.87 - ETA: 2s - loss: 0.0265 - accuracy: 0.87 - ETA: 2s - loss: 0.0265 - accuracy: 0.87 - ETA: 2s - loss: 0.0264 - accuracy: 0.87 - ETA: 2s - loss: 0.0265 - accuracy: 0.87 - ETA: 2s - loss: 0.0265 - accuracy: 0.87 - ETA: 2s - loss: 0.0265 - accuracy: 0.87 - ETA: 2s - loss: 0.0266 - accuracy: 0.87 - ETA: 2s - loss: 0.0266 - accuracy: 0.87 - ETA: 2s - loss: 0.0266 - accuracy: 0.87 - ETA: 2s - loss: 0.0266 - accuracy: 0.87 - ETA: 2s - loss: 0.0266 - accuracy: 0.87 - ETA: 2s - loss: 0.0266 - accuracy: 0.87 - ETA: 2s - loss: 0.0266 - accuracy: 0.87 - ETA: 2s - loss: 0.0266 - accu